In [20]:
from promptflow import tool
import json
import requests
import pandas as pd

def get_current_lat_long(address):
    url = f"https://geocode.maps.co/search"
    params = {
        "q": address
    }
    response = requests.get(url,params)
    response_json = response.json()
    print(response_json)
    if response_json:
        return response_json[0]["lat"], response_json[0]["lon"]
    else:
        return None

def get_current_weather(latitude,longitude):
    url = f"https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,windspeed_10m",
        "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m"
    }
    response = requests.get(url,params=params)
    return response.json()

# def get_current_weather(location, unit):
#     """Get the current weather in a given location"""
#     weather_info = {
#         "location": location,
#         "temperature": "72",
#         "unit": unit,
#         "forecast": ["sunny", "windy"],
#     }
#     return weather_info
import json
import pandas as pd
import requests

def get_n_day_weather_forecast(latitude, longitude, num_days):
    """Get historical weather for the given num_days. Limit num_days to less than 5"""
    url = f"https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "pastdays": num_days,
        "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m"
    }
    response = requests.get(url, params=params)
    # Create Pandas dataframe from Data
    df = pd.DataFrame(response.json()["hourly"])
    df["time"] = pd.to_datetime(df["time"])
    df = df.set_index("time")  # Set time column as index
    daily_summary = df.groupby(pd.Grouper(freq="D")).agg(["mean", "std"])
    daily_summary = df.describe()
    print(daily_summary.to_json(  orient="index"))

    # daily_summary_dict_str_keys = {}
    # for key, value in daily_summary_dict.items():
    #     daily_summary_dict_str_keys[str(key)] = value
    # # Convert the dictionary to a JSON string with indentation for readability
    # print(daily_summary_dict_str_keys)
    # daily_summary_json = json.dumps(daily_summary_dict_str_keys, indent=4)
    # return daily_summary_json

# def get_n_day_weather_forecast(location, format, num_days):
#     """Get next num_days weather in a given location"""
#     weather_info = {
#         "location": location,
#         "temperature": "60",
#         "format": format,
#         "forecast": ["rainy"],
#         "num_days": num_days,
#     }
#     return weather_info


@tool
def run_function(response_message: dict) -> str:
    if "function_call" in response_message:
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])
        print(function_args)
        result = globals()[function_name](**function_args)
    else:
        print("No function call")
        if isinstance(response_message, dict):
            result = response_message["content"]
        else:
            result = response_message
    return result


In [21]:
# get_current_lat_long("singapore")
get_n_day_weather_forecast(1.3521,103.8198,5)

{"count":{"temperature_2m":168.0,"relativehumidity_2m":168.0,"windspeed_10m":168.0},"mean":{"temperature_2m":28.2845238095,"relativehumidity_2m":79.8214285714,"windspeed_10m":5.3041666667},"std":{"temperature_2m":1.8442332225,"relativehumidity_2m":10.691269833,"windspeed_10m":3.0536981441},"min":{"temperature_2m":25.8,"relativehumidity_2m":46.0,"windspeed_10m":0.4},"25%":{"temperature_2m":26.875,"relativehumidity_2m":72.75,"windspeed_10m":2.975},"50%":{"temperature_2m":27.85,"relativehumidity_2m":82.0,"windspeed_10m":4.3},"75%":{"temperature_2m":29.6,"relativehumidity_2m":89.0,"windspeed_10m":7.9},"max":{"temperature_2m":33.8,"relativehumidity_2m":94.0,"windspeed_10m":14.1}}
